# **Convolutional Neural Network**


In [ ]:
!gdown --id '1wCdNcClcd2p5UeDi6XxNdiBedNaVn3fP' --output food-11.zip # 下載資料集
!unzip food-11.zip # 解壓縮  

In [ ]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

#Read image
利用 OpenCV (cv2) 讀入照片並存放在 numpy array 中

In [ ]:
def readfile(path, label):
    # label 是一個 boolean variable，代表需不需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(128, 128))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [ ]:
#分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = './food-11'
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347


# Dataset


In [ ]:
#training 時做 data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), #隨機將圖片水平翻轉
    transforms.RandomRotation(15), #隨機旋轉圖片
    transforms.ColorJitter(brightness = 0.1, contrast = 0.1, saturation = 0.1, hue = 0.1), # 改變圖片色調
    transforms.ToTensor(), #將圖片轉成 Tensor，並把數值normalize到[0,1](data normalization)
]) # 把要做的transform包在一起
#testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset): # 繼承了Dataset
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [ ]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform)
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Model

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        #torch.nn.MaxPool2d(kernel_size, stride, padding)
        #input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            #nn.Dropout2d(0.25),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 1024, 3, 1, 1), # [1024, 8, 8]
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [1024, 4, 4]

            nn.Conv2d(1024, 512, 3, 1, 1), # [512, 4, 4]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*2*2, 1024),
            #nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )
    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# Training

使用training set訓練，並使用validation set尋找好的參數

In [ ]:
model = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer 使用 Adam
# 在影像辨識上 SGD + Momentum 的表現通常比 Adam 好
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader):
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值
        print(train_pred)
        print(train_pred.shape)
        print(data[1])
        print(data[1].shape)
        break

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))
    break

tensor([[-0.1359, -0.0973,  0.1542,  ...,  0.1082, -0.1881,  0.1155],
        [-0.2284, -0.0867,  0.1552,  ..., -0.0445, -0.2068,  0.0539],
        [-0.1718, -0.1641,  0.1454,  ...,  0.0070, -0.1701,  0.0737],
        ...,
        [-0.1601, -0.1797,  0.1616,  ...,  0.0767, -0.2360,  0.0532],
        [-0.1655, -0.1881,  0.0545,  ..., -0.0655, -0.2092, -0.0081],
        [-0.0852, -0.1376,  0.1911,  ...,  0.0556, -0.1233, -0.0367]],
       device='cuda:0', grad_fn=<AddmmBackward>)
torch.Size([128, 11])
tensor([ 8,  4,  8,  2,  2,  9,  9, 10,  9,  6,  3,  5,  9,  5,  1,  8,  0,  2,
         4,  8,  5,  1,  3,  2,  5,  8,  9,  1,  4,  2,  0,  0,  2,  9,  4,  9,
         8,  5,  9,  2,  8,  9,  0,  6,  8,  8,  5, 10,  8,  8,  5,  0,  8,  0,
         2,  0,  3,  9,  5,  2,  6,  1,  9,  2,  3,  3,  2,  8,  1, 10, 10,  8,
         0,  9,  2,  3,  2,  3,  2,  4,  2,  4,  3,  3,  6, 10,  8,  5,  5,  4,
         9,  9,  9,  8, 10,  3,  0,  9,  5,  3,  0,  2,  2,  9,  8,  0,  2,  7,
         5,  5,

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

val_set = ImgDataset(val_x, transform=test_transform)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(val_loader):
        val_pred = model(data.cuda())
        val_label = np.argmax(val_pred.cpu().data.numpy(), axis=1) # 回傳最大值的index
        for y in val_label:
            prediction.append(y)
prediction = np.asarray(prediction)

In [ ]:
cm = confusion_matrix(val_y, prediction)

In [ ]:
len(prediction)

3430

In [ ]:
cm # 上方標籤為實際分類，左方標籤為預測分類

array([[ 64,  13,  42,  28,  26,  82,   2,   2,  51,  41,  11],
       [  1,  50,  46,   0,   3,   6,   0,   3,  11,  23,   1],
       [  6,  14, 254,   3,  10,  96,   0,   0,  55,  52,  10],
       [  7,   8,  31, 137,   9,  39,   1,   1,  54,  35,   5],
       [  0,   2,  43,   1, 168,  45,   0,   4,  41,  18,   4],
       [  0,   1,  13,   1,   7, 376,   0,   0,  39,   6,   6],
       [  3,   0,   2,   3,   1,  21,  62,   5,   3,  39,   8],
       [  4,   2,   4,   0,   2,   1,  10,  49,   5,   6,  13],
       [  0,   2,  22,   1,   3,  25,   0,   0, 274,  11,   9],
       [  0,   5,  14,   2,   0,   6,   0,   0,  14, 459,   0],
       [  0,   4,   5,   0,   0,   9,   0,   0,  17,   1, 196]])

In [ ]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

In [ ]:
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 120

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

# Apex

In [ ]:
%%writefile setup.sh
# 嘗試混合精度訓練
git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform)
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [ ]:
from apex import amp
model = Classifier().cuda()
loss_func = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 30
model, optimizer = amp.initialize(model, optimizer, opt_level="O1") # O1指定混合精度訓練
# 用fp16取代float可以減少儲存空間並加速訓練，然而fp16的範圍比float狹窄許多，尤其容易出現underflow的問題，因此做動態損失放大 scale_loss 
# 然而放大損失的過程中亦可能有overflow的問題，此時的做法是跳過目前步驟，選擇較小的放大倍數在執行一次
# Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0 
# 訓練初期容易有此問題，因為apex會嘗試較大的放大倍數
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)    
    for i, data in enumerate(train_loader):
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss_func(train_pred, data[1].cuda())
        with amp.scale_loss(batch_loss, optimizer) as scaled_loss:
          scaled_loss.backward()
        optimizer.step()
        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())
            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()
        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

# Testing

In [ ]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

# Saving model and results

In [ ]:
#將結果寫入 csv 檔
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))

In [ ]:
from google.colab import files
files.download('predict.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
model_save_name = 'classifier.pt' # 107 epoch
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model_best.state_dict(), path)

# Downloading model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 53

In [ ]:
path = "/content/gdrive/My Drive/classifier.pt"
model_best.load_state_dict(torch.load(path))

<All keys matched successfully>

# Keep training

In [ ]:
# 正確率爬不上去時修改learning rate，加速訓練使training accuracy為1
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.0001) # optimizer 使用 Adam
num_epoch = 40
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

[001/040] 56.72 sec(s) Train Acc: 0.995337 Loss: 0.000114
[002/040] 57.02 sec(s) Train Acc: 0.997443 Loss: 0.000054
[003/040] 57.32 sec(s) Train Acc: 0.998270 Loss: 0.000032
[004/040] 57.61 sec(s) Train Acc: 0.999323 Loss: 0.000023
[005/040] 57.84 sec(s) Train Acc: 0.999248 Loss: 0.000018
[006/040] 57.59 sec(s) Train Acc: 0.999173 Loss: 0.000016
[007/040] 57.58 sec(s) Train Acc: 0.999474 Loss: 0.000016
[008/040] 57.65 sec(s) Train Acc: 0.999398 Loss: 0.000011
[009/040] 57.67 sec(s) Train Acc: 0.999474 Loss: 0.000010
[010/040] 57.49 sec(s) Train Acc: 0.999398 Loss: 0.000013
[011/040] 57.78 sec(s) Train Acc: 0.999774 Loss: 0.000008
[012/040] 57.61 sec(s) Train Acc: 1.000000 Loss: 0.000004
[013/040] 57.62 sec(s) Train Acc: 0.999774 Loss: 0.000006
[014/040] 57.67 sec(s) Train Acc: 0.999699 Loss: 0.000006
[015/040] 57.64 sec(s) Train Acc: 0.999624 Loss: 0.000006
[016/040] 57.88 sec(s) Train Acc: 0.999774 Loss: 0.000011
[017/040] 57.55 sec(s) Train Acc: 0.999549 Loss: 0.000007
[018/040] 57.6